## Vector-valued function with Random Fourier Features

### Imports

In [132]:
import numpy as np
from numpy.random import randn

### Values

In [133]:

dimension = 2
sigma = 10
sigma2 = 2*sigma**2

# Random Fourier features
m = 20 # number of samples
w = np.array([randn(m) / sigma, randn(m) / sigma]) # w is the same size as the dimensions times the number of samples
b = np.random.rand(m)*2*np.pi # b is the same size as number of samples

n = 100 # number of points

x1 = randn(n)/2
x2 =  randn(n)/2
x_vector = np.array([x1, x2])
# x_vector = np.block([x1,x2])

y1 = np.cos(x1) + randn(n) * 1e-1
y2 = np.sin(x2) + randn(n) * 1e-1


#y1 = 1+np.sin(x1/10) + x1**2
#y2 = 0.5 * x2**2 + np.exp(x2)

y = np.block([y1, y2])
print(y)

#print(y1, y2)
lam = 0.000001 # Regularisaton parameter

[ 0.87991803  0.81758991  0.90942762  0.8816141   0.52765491  0.84829082
  0.95678949  0.61917927  0.95540762  1.0000273   1.06843323  0.91146028
  1.09523667  0.93391261  0.6093853   0.98173218  0.75010272  0.80713659
  1.0435593   1.05900995  1.13680307  0.73311815  1.05390371  0.84588822
  0.93557145  0.91040602  0.87954762  1.23253247  0.97031804  0.96920075
  1.0087039   0.80136741  0.93425545  0.8492191   0.70387757  1.00448907
  0.98302337  1.06461388  1.14466577  0.96905019  0.91663385  0.90794544
  0.94225169  0.70868758  1.07695822  0.9304767   0.79242186  0.82727078
  0.96452969  0.96839596  0.87251466  0.92681911 -0.37474965  0.89830483
  0.98184655  0.55877617  0.5870954   0.90695752  1.08339604  0.9996478
  1.10264327  1.02140428  1.136425    1.03115381  0.8627832   0.94709002
  0.82195847  0.89821482  0.95571792  0.82935635  0.9794097   0.92812674
  0.95538672  0.97522566  0.7156126   0.53901925  0.98722121  0.83329275
  0.96507273  1.08137324  0.90808498  0.8510124   0.

### Gaussian Kernel

In [134]:
def k_gauss(x, z):
    k = np.exp(-((np.linalg.norm(x-z))**2)/(2*sigma**2))
    return k

def k_matrix(x, z, dim):
    k = k_gauss(x, z)
    k_matrix = k *np.eye(dim)
    return k_matrix

def K_gauss(x, n, dim):
    K = np.zeros((dim*n, dim*n))
    for i in range(0, n):
        for j in range(0, n):
            k = k_gauss(x[:,i], x[:,j])
            K[i*dim][j*dim] = k
            K[i*dim+1][j*dim+1] = k
    return K

K = K_gauss(x_vector, n, dimension)
print(K)

[[1.         0.         0.99245651 ... 0.         0.99689507 0.        ]
 [0.         1.         0.         ... 0.99253008 0.         0.99689507]
 [0.99245651 0.         1.         ... 0.         0.99853284 0.        ]
 ...
 [0.         0.99253008 0.         ... 1.         0.         0.99815652]
 [0.99689507 0.         0.99853284 ... 0.         1.         0.        ]
 [0.         0.99689507 0.         ... 0.99815652 0.         1.        ]]


#### Alpha

In [135]:
# Returns a vector with length dim * n
def alpha(x, y, lam, n, dim):
    K = K_gauss(x, n, dim)
    alpha_vector = np.linalg.inv((K + n*lam*np.eye(dim*n))) @ y
    alpha_reshape = [alpha_vector[::2], alpha_vector[1::2]]
    return np.array(alpha_reshape)

alpha_ = alpha(x_vector, y, lam, n, dimension)

#### Estimated function

In [136]:
def estiamted_function(alpha, x, n, dim):
    function = np.zeros((2, n))
    for i in range(n):
        sum1 = np.zeros((2))
        x_i = x[:, i]
        for j in range(n):
            k = k_gauss(x_i, x[:, j])
            kernel = k*np.eye(dim)
            sum1 = np.add(sum1, kernel@alpha[:, j])
        function[:, i] = sum1 # Blir ca halvparten så liten :((
    return function

estiamted_function(alpha_, x_vector, n, dimension)


array([[ 0.45424941,  0.27995055,  0.15879048,  0.52078615,  0.32292682,
         0.44995192,  0.56018479,  0.30830179,  0.38008741,  0.44392909,
         0.4561405 ,  0.38540497,  0.52080323,  0.36467385,  0.37061752,
         0.43659467,  0.452978  ,  0.43018375,  0.56730928,  0.44144696,
         0.38945723,  0.43376254,  0.35962504,  0.43383223,  0.43204219,
         0.44682486,  0.44790381,  0.45630041,  0.40693163,  0.33949793,
         0.4394549 ,  0.54490163,  0.57349607,  0.37155342,  0.86202811,
         0.44431198,  0.37573848,  0.52006605,  0.35484455,  0.45375493,
         0.46615525,  0.72960539,  0.56340968,  0.51787128,  0.50977682,
         0.30062658,  0.6016554 ,  0.41286257,  0.13965074,  0.32137165,
         0.47977884,  0.44122346,  0.44856664,  0.32009758,  0.34956879,
        -0.09127949,  0.32931836,  0.33073601,  0.45752036,  0.39008173,
         0.44432263,  0.47120172,  0.51710409,  0.48499555,  0.43378541,
         0.46145961,  0.45613084,  0.42648185,  0.4

#### Plot

### With Random Fourier Features

#### Psi

In [137]:
def psi(x, w, b):
    return np.sqrt(2) * np.cos(w.T@x + b)

def psi_vector(x, w, b, m):
    psi_vector = np.zeros(m)
    for i in range(m):
        psi_vector[i] = psi(x, w[:, i], b[i])
    return np.array(psi_vector)

def capital_psi(x, w,b, m, dim):
    psi = psi_vector(x,w,b,m)
    return np.kron(psi, np.eye(dim))

capital_psi(x_vector[:, 1], w, b, m, dimension)

array([[ 0.15614591,  0.        ,  0.57706183,  0.        ,  1.39354577,
         0.        , -1.05750543, -0.        , -0.81752137, -0.        ,
         1.31028132,  0.        ,  0.77330295,  0.        , -0.81756843,
        -0.        ,  0.8893866 ,  0.        ,  0.9597386 ,  0.        ,
         0.30156887,  0.        ,  0.21520222,  0.        ,  0.78092239,
         0.        , -1.28437295, -0.        , -1.12560319, -0.        ,
         1.41407554,  0.        ,  0.6648447 ,  0.        , -1.38579064,
        -0.        ,  1.39945654,  0.        ,  0.5440162 ,  0.        ],
       [ 0.        ,  0.15614591,  0.        ,  0.57706183,  0.        ,
         1.39354577, -0.        , -1.05750543, -0.        , -0.81752137,
         0.        ,  1.31028132,  0.        ,  0.77330295, -0.        ,
        -0.81756843,  0.        ,  0.8893866 ,  0.        ,  0.9597386 ,
         0.        ,  0.30156887,  0.        ,  0.21520222,  0.        ,
         0.78092239, -0.        , -1.28437295, -0.

#### Phi

In [138]:
def phi(x, w, b, n, m, dim):
    phi = np.zeros((dim*n, dim*m))
    for i in range(0, n):
        psi = capital_psi(x[:,i], w, b, m, dim)
        phi[dim*i] = psi[0]
        phi[dim*i+1] = psi[1]
    return phi

phi_ = phi(x_vector, w, b, n, m, dimension)
phi_

array([[0.37612072, 0.        , 0.65128392, ..., 0.        , 0.70553953,
        0.        ],
       [0.        , 0.37612072, 0.        , ..., 1.3557407 , 0.        ,
        0.70553953],
       [0.15614591, 0.        , 0.57706183, ..., 0.        , 0.5440162 ,
        0.        ],
       ...,
       [0.        , 0.46857797, 0.        , ..., 1.401054  , 0.        ,
        0.64517486],
       [0.31831125, 0.        , 0.60365216, ..., 0.        , 0.62499539,
        0.        ],
       [0.        , 0.31831125, 0.        , ..., 1.3895833 , 0.        ,
        0.62499539]])

#### Alpha

In [139]:
def alpha_approx(phi, y, n, dim, m, lam):
    alpha = np.linalg.inv(phi.T @ phi + lam*n*np.eye(dim*m)) @ (phi.T@y)
    return alpha

alpha_approximation = alpha_approx(phi_, y, n, dimension, m, lam)
alpha_approximation.shape

(40,)

#### Function Approximation

In [147]:
def function_approximation(alpha, x, n, dim):
    f = np.zeros((dim, n))
    for i in range(n):
        f[:, i] = capital_psi(x[:, i], w, b, m, dim) @ alpha
    return f


def func_2(alpha, phi):
    return phi @ alpha

a = function_approximation(alpha_approximation, x_vector, n, dimension)
print(a)
print(func_2(alpha_approximation, phi_))

[ 0.44950019  0.42928032  0.2703557   0.52751892  0.13029567  0.42394395
  0.55184987  0.40070267  0.34343108  0.35954785  0.43777879  0.42966326
  0.58661001  0.6766885   0.30825172  0.39435182  0.37866403  0.48750554
  0.43815923  0.47819211  0.47161831  0.66784452  0.38679058  0.43735095
  0.54131368  0.39896158  0.36290233  0.50316137  0.38855763  0.38352736
  0.41086015  0.3950673   0.4347658   0.49619116  0.40444914  0.42123954
  0.58336389  0.63479444  0.44798559  0.42267193  0.38839261  0.48692976
  0.43094392  0.48130531  0.3570852   0.49190709  0.41064177  0.44543448
  0.43714264  0.55146046  0.43549675  0.46580853  0.43781434  0.47312312
  0.46075594  0.42141865  0.4109601   0.43003893  0.33559913  0.45243111
  0.41947271  0.45602174  0.55047416  0.60361044  0.59554498  0.6547933
  0.37085397  0.51496595  0.91378975  0.81908285  0.42748585  0.43010346
  0.38174087  0.41283274  0.53241842  0.39326816  0.35171654  0.4787093
  0.44768434  0.49867675  0.45806876  0.51408316  0.8